In [2]:
# Validate GPU
import torch
assert(torch.cuda.is_available())
assert(torch.cuda.device_count() > 0)
assert(torch.cuda.current_device() == 0)
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

NVIDIA GeForce GTX 1080 Ti


In [3]:
# Load dataset
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("Length of data in characters: ", len(text), "\n---------------------\n")
print(text[:100])

Length of data in characters:  1115394 
---------------------

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
# Look at all the unique characters within the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# Encoding / decoding function are just lookup tables from the string / number and vice versa
strToNum = { ch:i for i,ch in enumerate(chars) }
numToStr = { i:ch for i,ch in enumerate(chars) }
encode = lambda string : [strToNum[char] for char in string]
decode = lambda num_list : ''.join([numToStr[i] for i in num_list])

startString = "Hey... whats up?"
encoded = encode(startString)
decoded = decode(encoded)

print(startString)
print(encoded)
print(decoded)

Hey... whats up?
[20, 43, 63, 8, 8, 8, 1, 61, 46, 39, 58, 57, 1, 59, 54, 12]
Hey... whats up?


In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [8]:
# Separate data into train/test split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8 # Can also be called context length
train_data[:block_size + 1] # Add plus one

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
# Example. It learns that given some series of input, the next character will be TARGET
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target: {target}")

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [11]:
torch.manual_seed(1337) # Add a seed to reproduce the same results as karpathy
batch_size = 4 # How many independent sequences we process in parallel
block_size = 8 # What is the maximum context length for predictions

def get_batch(split):
    # Generate a small batch of data of inputs x and targets y
    batch_data = train_data if split == 'train' else val_data
    ix = torch.randint(len(batch_data) - block_size, (batch_size,))
    new_x = torch.stack([batch_data[i : i + block_size] for i in ix])
    new_y = torch.stack([batch_data[i + 1: i + block_size + 1] for i in ix])
    return new_x, new_y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('----')

for b in range(batch_size): # Batch dimension
    for t in range(block_size): # Time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'When input is {context.tolist()} the target is: {target}')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
When input is [24] the target is: 43
When input is [24, 43] the target is: 58
When input is [24, 43, 58] the target is: 5
When input is [24, 43, 58, 5] the target is: 57
When input is [24, 43, 58, 5, 57] the target is: 1
When input is [24, 43, 58, 5, 57, 1] the target is: 46
When input is [24, 43, 58, 5, 57, 1, 46] the target is: 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] the target is: 39
When input is [44] the target is: 53
When input is [44, 53] the target is: 56
When input is [44, 53, 56] the target is: 1
When input is [44, 53, 56, 1] the target is: 58
When input is [44, 53, 56, 1, 58] 

In [12]:
# Bi-gram language model definition. One of the simplest language models available

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    token_embedding_table: nn.Embedding

    def __init__(self, input_vocab_size):
        super().__init__()
        # Each token directly reads off the logits for the next token from a lookup table
        print(input_vocab_size)
        self.token_embedding_table = nn.Embedding(input_vocab_size, input_vocab_size)

    def forward(self, idx, targets = None):
        # Idx and targets are both (B, T) tensors of integers
        new_logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            new_loss = None
        else:
            b, t, c = new_logits.shape
            new_logits = new_logits.view(b * t, c)
            targets = targets.view(b * t)
            new_loss = F.cross_entropy(new_logits, targets)


        return new_logits, new_loss

    def generate(self, idx, max_new_tokens):
        # Idx is a (B, T) array of indicies in the current context
        for _ in range(max_new_tokens):
            # Get the predictions
            new_logits, new_loss = self(idx)

            # Focus only on the last time step
            new_logits = new_logits[:, -1, :] # Get last element of the time dimension which is (B, C)

            # Apply softmax to get the probabilities
            probs = F.softmax(new_logits, dim=1) # Becomes (B, C)

            # Sample 1 sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # Becomes (B, 1)

            # Append sampled index to the running sequence
            idx = torch.cat([idx, idx_next], dim=1) # Becomes (B, T+1)
        return idx



bigram_model = BigramLanguageModel(vocab_size)
logits, loss = bigram_model(xb, yb)
print(logits.shape, loss)

test_idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(bigram_model.generate(test_idx, max_new_tokens=100)[0].tolist()))


65
torch.Size([32, 65]) tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [13]:
# Create an optimizer
optimizer = torch.optim.AdamW(bigram_model.parameters(), lr=1e-3)

In [14]:
batch_size = 32
for steps in range(10000):
    # Sample a batch of data
    xb, yb = get_batch('train')

    # Forward pass
    logits, loss = bigram_model(xb, yb)

    # Evaluate the loss
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.382369041442871


In [15]:
test_idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(bigram_model.generate(test_idx, max_new_tokens=100)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulsee


# The matematical trick in self-attention

In [23]:
torch.manual_seed(1337)
b, t, c = 4, 8, 2
x = torch.randn(b, t, c)
x.shape

torch.Size([4, 8, 2])

In [24]:
# We want x[b, t] = mean_{i<=t} x[b, i]
x_bow = torch.zeros((b, t, c))
for batch in range(b):
    for time in range(t):
        x_prev = x[batch, :time + 1] # (t, c)
        x_bow[batch, time] = torch.mean(x_prev, 0) # Averages out the time dimension

In [25]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [26]:
x_bow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [28]:
# We can use matrix multiplication with triangular matricies instead of the for loop
# We create a lower triangular matrix with ones on the diagonal
torch.manual_seed(42)
a = torch.tril(torch.ones((3, 3)))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b

print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [37]:
# We want x[b, t] = mean_{i<=t} x[b, i]
wei = torch.tril(torch.ones((t, t)))
wei = wei / torch.sum(wei, 1, keepdim=True)
x_bow_2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)

In [38]:
torch.allclose(x_bow, x_bow_2)

True

In [43]:
# Version 3: Use softmax
tril = torch.tril(torch.ones((t, t)))
wei = torch.zeros((t, t))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1) # Exponentiates every element, then normalizes

x_bow_3 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(x_bow, x_bow_3)

True

In [71]:
# Version 4: self-attention!
# Different tokens will find different other tokens interesting. EG: Vowels care about previous consonants. Solved by self attention
# Every single node emits vectors: query (What am i looking for), key (What do i contain). Dot product of query and key which becomes wei. If these values align, we learn more about the token
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# Let's see a single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)
k = key(x)      # (B, T, 16)
q = query(x)    # (B, T, 16)
wei  = q @ k.transpose(-2, -1) * head_size ** -0.5# (B, T, 16) @ (B, 16, T) ----> (B, T, T)

tril = torch.tril(torch.ones((T, T)))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1) # Exponentiates every element, then normalizes

v = value(x)
x_bow_4 = wei @ v # (B, T, T) @ (B, T, C) ----> (B, T, C)
x_bow_4.shape

torch.Size([4, 8, 16])

In [72]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3966, 0.6034, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3069, 0.2892, 0.4039, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3233, 0.2175, 0.2443, 0.2149, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1479, 0.2034, 0.1663, 0.1455, 0.3369, 0.0000, 0.0000, 0.0000],
        [0.1259, 0.2490, 0.1324, 0.1062, 0.3141, 0.0724, 0.0000, 0.0000],
        [0.1598, 0.1990, 0.1140, 0.1125, 0.1418, 0.1669, 0.1061, 0.0000],
        [0.0845, 0.1197, 0.1078, 0.1537, 0.1086, 0.1146, 0.1558, 0.1553]],
       grad_fn=<SelectBackward0>)